# (Py)Spark Exercises

_Giovanni Simonini - Giuseppe Fiameni_

## Spark vs Pyspark?

Spark is written in Scala. The 'native' API is in Scala.

_Pyspark_ is a lightweight wrapper around the native API. (You can see its implementation [here](https://github.com/apache/spark/tree/master/python/pyspark).)

---

![](http://i.imgur.com/YlI8AqEl.png)

[source](https://cwiki.apache.org/confluence/display/SPARK/PySpark+Internals)

---

## Introduction to the (py)Spark

#### Warm up by creating an RDD (Resilient Distributed Dataset) named `pagecounts` from the input files.

Let's create the SparkContext `sc`.

In [ ]:
from pyspark import SparkConf, SparkContext
conf = SparkConf()
sc = SparkContext(conf=conf)
'''
if a spark context is already available this snippet fails
'''
#PATH = "/notebooks/cineca/"

In [ ]:
sc

In [ ]:
pagecounts = sc.textFile("./data/pagecounts")

# ! head -n 1000 ./data/pagecounts | cut -d' ' -f 2 | uniq | sort

Let's take a peek at the data. You can use the take operation of an RDD to get the first N records. Here, N = 10.

In [ ]:
pagecounts.take(10)

Unfortunately this is not very readable because `take()` returns an array and Python simply prints the array with each element separated by a comma.
We can make it prettier by traversing the array to print each record on its own line.

In [ ]:
for x in pagecounts.take(10):
    print(x)

Let's see how many records in total are in this data set (this command will take a while, so read ahead while it is running).

In [ ]:
pagecounts.count()

## Cache in memory
Recall from above when we described the format of the data set, that the second field is the "project code" and contains information about the language of the pages.

For example, the project code "en" indicates an English page.
   Let's derive an RDD containing only English pages from `pagecounts`.
   This can be done by applying a filter function to `pagecounts`.
   For each record, we can split it by the field delimiter (i.e. a space) and get the second field-– and then compare it with the string "en".

   To avoid reading from disks each time we perform any operations on the RDD, we also __cache the RDD into memory__.
    This is where Spark really starts to to shine.

In [ ]:
enPages = pagecounts.filter(lambda x: x.split(" ")[1] == "en").cache()

When you type this command into the Spark shell, Spark defines the RDD, but because of lazy evaluation, no computation is done yet.
   Next time any action is invoked on `enPages`, Spark will cache the data set in memory across the workers in your cluster.

**How many records are there for English pages?**

In [ ]:
enPages.count()

The first time this command is run, similar to the last count we did, it will take 2 - 3 minutes while Spark scans through the entire data set on disk.
   __But since `enPages` was marked as "cached" in the previous step, if you run count on the same RDD again, it should return an order of magnitude faster__.


In [ ]:
enPages.count()

### RDD Persistency levels

*Storage Level* | *Meaning*
--- | ---
MEMORY_ONLY | Store RDD as deserialized Java objects in the JVM. If the RDD does not fit in memory, some partitions will not be cached and will be recomputed on the fly each time they're needed. This is the default level.
MEMORY_AND_DISK | Store RDD as deserialized Java objects in the JVM. If the RDD does not fit in memory, store the partitions that don't fit on disk, and read them from there when they're needed.
MEMORY_ONLY_SER | Store RDD as serialized Java objects (one byte array per partition). This is generally more space-efficient than deserialized objects, especially when using a fast serializer, but more CPU-intensive to read.
MEMORY_AND_DISK_SER | Similar to MEMORY_ONLY_SER, but spill partitions that don't fit in memory to disk instead of recomputing them on the fly each time they're needed.
DISK_ONLY|	Store the RDD partitions only on disk.
MEMORY_ONLY_2, MEMORY_AND_DISK_2, etc. | Same as the levels above, but replicate each partition on two cluster nodes.

> `RDD.persist(MEMORY_ONLY)`

**Let's try something fancier.**

Generate a histogram of total page views on Wikipedia English pages for the date range represented in our dataset (May 5 to May 7, 2009). The high level idea of what we'll be doing is as follows. First, we generate a key value pair for each line; the key is the date (the first eight characters of the first field), and the value is the number of pageviews for that date (the fourth field).

In [ ]:
enTuples = enPages.map(lambda x: x.split(" "))
enKeyValuePairs = enTuples.map(lambda x: (x[0][:8], int(x[3])))
enKeyValuePairs.count()

Next, we shuffle the data and group all values of the same key together.
Finally we sum up the values for each key.
There is a convenient method called `reduceByKey` in Spark for exactly this pattern.

Note that the second argument to `reduceByKey` determines the number of reducers to use.
By default, Spark assumes that the reduce function is commutative and associative and applies combiners on the mapper side.
Since we know there is a very limited number of keys in this case (because there are only 3 unique dates in our data set), let's use only one reducer.

In [ ]:
enKeyValuePairs.reduceByKey(lambda x, y: x + y, 1).collect()

The `collect` method at the end converts the result from an RDD to an `array`.

We can combine the previous three commands into one:

In [ ]:
enPages.map(lambda x: x.split(" "))\
        .map(lambda x: (x[0][:8], int(x[3])))\
        .reduceByKey(lambda x, y: x + y, 1)\
        .collect()

Suppose we want to find pages that were viewed more than 200,000 times during the three days covered by our dataset. Conceptually, this task is similar to the previous query. But, given the large number of pages the new task is very expensive. We are doing an expensive group-by with a lot of network shuffling of data.

To recap:
   1. _first we split each line of data into its fields_
   2. _we extract the fields for page name and number of page views_
   3. _we reduce by key again, this time with 40 reducers_
   4. _then we filter out pages with less than 200,000 total views over our time window represented by our dataset_

In [ ]:
enPages.map(lambda x: x.split(" "))\
        .map(lambda x: (x[2], int(x[3])))\
        .reduceByKey(lambda x, y: x + y, 40)\
        .filter(lambda x: x[1] > 200000)\
        .map(lambda x: (x[1], x[0]))\
        .collect()

There is no hard and fast way to calculate the optimal number of reducers for a given problem; you will build up intuition over time by experimenting with different values.


You can explore the full RDD API by browsing the [Java/Scala](http://www.cs.berkeley.edu/~pwendell/strataconf/api/core/index.html#spark.RDD) or [Python](http://www.cs.berkeley.edu/~pwendell/strataconf/api/pyspark/index.html) API docs.

## PySpark API

Let's look at some PySpark API.

> By clicking on the images you can directly access to the documentation.



<a href="http://spark.apache.org/docs/1.2.0/api/python/pyspark.html#pyspark.RDD.map">
<img align=center src="./pyspark_API/images/pyspark-page3.svg" width=400 height=400 />
</a>



__`map(func)`__	Return a new distributed dataset formed by passing each element of the source through a function func.

In [ ]:
# map
x = sc.parallelize([1,2,3]) # sc = spark context, parallelize creates an RDD from the passed object
print(x.collect())  # collect copies RDD elements to a list on the driver

y = x.map(lambda x: (x,x**2))
print(y.collect())

<a href="http://spark.apache.org/docs/1.2.0/api/python/pyspark.html#pyspark.RDD.flatMap">
<img align=center src="./pyspark_API/images/pyspark-page4.svg" width=400 height=400/>
</a>

__`flatMap(func)`__	Similar to map, but each input item can be mapped to 0 or more output items (so func should return a Seq rather than a single item).


In [ ]:
# flatMap
x = sc.parallelize([1,2,3])
print(x.collect())

def f(x):
    return [100*x, x**2]
print(x.map(f).collect()) # Map

y = x.flatMap(
    lambda x: (x, 100*x, x**2)) # this lambda yields 2 elements for each element of x
print(y.collect())

<a href="http://spark.apache.org/docs/1.2.0/api/python/pyspark.html#pyspark.RDD.filter">
<img align=center src="./pyspark_API/images/pyspark-page8.svg" width=400 height=400 />
</a>

__`filter(func)`__	Return a new dataset formed by selecting those elements of the source on which func returns true.

In [ ]:
# filter
x = sc.parallelize([1,2,3,4,5,6,7,8,9])
print(x.collect())

y = x.filter(lambda x: x%2 == 1)  # filters out even elements
print(y.collect())

<a href="http://spark.apache.org/docs/1.2.0/api/python/pyspark.html#pyspark.RDD.reduce">
<img align=center src="./pyspark_API/images/pyspark-page23.svg" width=400 height=400/>
</a>

__`reduce(func)`__	Aggregate the elements of the dataset using a function func (which takes two arguments and returns one). The function should be commutative and associative so that it can be computed correctly in parallel.

In [ ]:
# reduce
x = sc.parallelize([1,2,3])
print(x.collect())

y = x.reduce(lambda obj, accumulated: obj + accumulated)  # computes a cumulative sum
print(y)

<a href="http://spark.apache.org/docs/1.2.0/api/python/pyspark.html#pyspark.RDD.reduceByKey">
<img align=center src="./pyspark_API/images/pyspark-page44.svg" width=400 height=400/>
</a>

__`reduceByKey(func, [numTasks])`__	When called on a dataset of (K, V) pairs, returns a dataset of (K, V) pairs where the values for each key are aggregated using the given reduce function func, which must be of type (V,V) => V. Like in groupByKey, the number of reduce tasks is configurable through an optional second argument.

In [ ]:
# reduceByKey
x = sc.parallelize([('B',1),('B',2),('A',3),('A',4),('A',5)])
print(x.collect())

y = x.reduceByKey(lambda agg, obj: agg + obj)
print(y.collect())

<a href="http://spark.apache.org/docs/1.2.0/api/python/pyspark.html#pyspark.RDD.sortBy">
<img align=center src="./pyspark_API/images/pyspark-page15.svg" width=400 height=400 />
</a>

__`sortByKey([ascending], [numTasks])`__	When called on a dataset of `(K, V)` pairs where K implements Ordered, returns a dataset of `(K, V)` pairs sorted by keys in ascending or descending order, as specified in the boolean ascending argument.

In [ ]:
# sortBy
x = sc.parallelize(['Cat','Apple','Bat'])
print(x.collect())

def keyGen(val):
    return val[0]

y = x.sortBy(keyGen)

print(y.collect())

<a href="http://spark.apache.org/docs/1.2.0/api/python/pyspark.html#pyspark.RDD.take">
<img align=center src="./pyspark_API/images/pyspark-page39.svg" width=400 height=400 />
</a>

__`take(n)`__ Return an array with the first n elements of the dataset.

In [ ]:
x = sc.parallelize([1,3,1,2,3])
print(x.collect())

y = x.take(num = 3)
print(y)

<a href="http://spark.apache.org/docs/1.2.0/api/python/pyspark.html#pyspark.RDD.distinct">
<img align=center src="./pyspark_API/images/pyspark-page9.svg" width=400 height=400 />
</a>

__`distinct([numTasks]))`__	Return a new dataset that contains the distinct elements of the source dataset.

In [ ]:
# distinct
x = sc.parallelize(['A','A','B','A','B'])
print(x.collect())

y = x.distinct()
print(y.collect())

# End of this chapter